# Resource Scheduling with Constraint Solver DSL

A functional, math-like DSL for constraint programming in Kotlin.

## Philosophy

- **Declarative over imperative**: Express *what* you want, not *how* to compute it
- **Functional composition**: Build complex constraints from simple, composable parts
- **Solver-agnostic**: The DSL abstracts away the underlying solver (CP-SAT)


## Setup

Load the solver library (includes DSL, Arrow, and OR-Tools bundled together).

**Build first:** `cd kotlin && ./gradlew :solver:shadowJar`


In [1]:
// Load the fat JAR with all dependencies bundled
@file:DependsOn("../kotlin/solver/build/libs/solver-all-0.1.0-SNAPSHOT.jar")

import arrow.core.*
import io.github.andreifilonenko.kpsat.dsl.*
import io.github.andreifilonenko.kpsat.solver.*

// Load OR-Tools native library
com.google.ortools.Loader.loadNativeLibraries()

println("Libraries loaded: DSL, Solver, Arrow, OR-Tools")


Libraries loaded: DSL, Solver, Arrow, OR-Tools


## Core Concepts

The DSL is built around **expressions** (`Expr`) that form an AST. You compose expressions using familiar operators, then the solver compiles them to constraints.


In [2]:
// Expressions are created using factory functions from the library
// expr(value) creates a constant, operators build the AST

// Constants
val a = expr(10L)
val b = expr(5L)

// Arithmetic: +, -, *, unary -
println("a + b = ${a + b}")
println("a - b = ${a - b}")
println("a * b = ${a * b}")
println("-a = ${-a}")

// Comparison: eq, neq, lt, leq, gt, geq (return boolean expressions)
println("\na eq b = ${a eq b}")
println("a lt b = ${a lt b}")

// Logical: and, or, not, implies
val p = a gt 5L
val q = b leq 10L
println("\np and q = ${p and q}")
println("p implies q = ${p implies q}")

// Conditional: iif(condition, ifTrue, ifFalse)
println("\niif(p, 100, 0) = ${iif(p, 100L, 0L)}")


a + b = Expr(node=Sum(operands=[Const(value=10), Const(value=5)]))
a - b = Expr(node=Sub(left=Const(value=10), right=Const(value=5)))
a * b = Expr(node=Prod(operands=[Const(value=10), Const(value=5)]))
-a = Expr(node=Neg(operand=Const(value=10)))

a eq b = Expr(node=Eq(left=Const(value=10), right=Const(value=5)))
a lt b = Expr(node=Lt(left=Const(value=10), right=Const(value=5)))

p and q = Expr(node=And(operands=[Gt(left=Const(value=10), right=Const(value=5)), Leq(left=Const(value=5), right=Const(value=10))]))
p implies q = Expr(node=Or(operands=[Not(operand=Gt(left=Const(value=10), right=Const(value=5))), Leq(left=Const(value=5), right=Const(value=10))]))

iif(p, 100, 0) = Expr(node=If(condition=Gt(left=Const(value=10), right=Const(value=5)), ifTrue=Const(value=100), ifFalse=Const(value=0)))


## Functional Aggregations

Use `sum`, `forAll`, `exists` to transform collections into constraints. These compose naturally with Kotlin's collection functions.


In [3]:
// Aggregations work with any Iterable
val items = listOf(1, 2, 3, 4, 5)

// sum: aggregate values
val totalCost = sum(items) { expr(it.toLong()) * expr(10L) }
println("sum with selector: $totalCost")

// forAll: universal quantifier (AND over collection)
val allPositive = forAll(items) { expr(it.toLong()) gt 0L }
println("forAll (item > 0): $allPositive")

// exists: existential quantifier (OR over collection)
val hasLarge = exists(items) { expr(it.toLong()) gt 3L }
println("exists (item > 3): $hasLarge")

// Compose with Kotlin collection functions
val filtered = items
    .filter { it > 2 }
    .map { expr(it.toLong()) }
    .let { sum(it) }
println("sum of filtered: $filtered")


sum with selector: Expr(node=Sum(operands=[Prod(operands=[Const(value=1), Const(value=10)]), Prod(operands=[Const(value=2), Const(value=10)]), Prod(operands=[Const(value=3), Const(value=10)]), Prod(operands=[Const(value=4), Const(value=10)]), Prod(operands=[Const(value=5), Const(value=10)])]))
forAll (item > 0): Expr(node=And(operands=[Gt(left=Const(value=1), right=Const(value=0)), Gt(left=Const(value=2), right=Const(value=0)), Gt(left=Const(value=3), right=Const(value=0)), Gt(left=Const(value=4), right=Const(value=0)), Gt(left=Const(value=5), right=Const(value=0))]))
exists (item > 3): Expr(node=Or(operands=[Gt(left=Const(value=1), right=Const(value=3)), Gt(left=Const(value=2), right=Const(value=3)), Gt(left=Const(value=3), right=Const(value=3)), Gt(left=Const(value=4), right=Const(value=3)), Gt(left=Const(value=5), right=Const(value=3))]))
sum of filtered: Expr(node=Sum(operands=[Const(value=3), Const(value=4), Const(value=5)]))


## Building Models with the Solver

The `ConstraintSolverBuilder` provides a functional, declarative API. Chain methods to define your model—no mutable state exposed.


In [4]:
// Pure functional model definition - no side effects until solve()
// The builder pattern chains immutably

val simpleModel = ConstraintSolverBuilder()
    .timeLimit(30)
    .numWorkers(4)
    
    // Variables: lambda with explicit scope parameter
    .variables { scope ->
        mapOf(
            "x" to scope.int("x", 0, 10),
            "y" to scope.int("y", 0, 10)
        )
    }
    
    // Hard constraint: must be satisfied
    .hard("sum_equals_15") { scope, vars ->
        (vars["x"]!! + vars["y"]!!) eq 15L
    }
    
    // Maximize the product
    .maximize("product", priority = 0) { scope, vars ->
        vars["x"]!! * vars["y"]!!
    }
    
    .build()

println("Model built (nothing solved yet - lazy evaluation)")


Model built (nothing solved yet - lazy evaluation)


## Solving and Handling Results

Call `solve()` to run the solver and get results.


In [5]:
// Solve returns a SolveResult
val result = simpleModel.solve()

// Pattern match on status
when (result.status) {
    SolveStatus.OPTIMAL -> {
        println("✓ Optimal solution found!")
        println("  Objective value: ${result.objectiveValue}")
    }
    SolveStatus.FEASIBLE -> {
        println("~ Feasible solution (may not be optimal)")
    }
    SolveStatus.INFEASIBLE -> {
        println("✗ No feasible solution exists")
    }
    else -> {
        println("? Status: ${result.status}")
    }
}

// Extract values
if (result.hasSolution) {
    val x = result.scope.value(result.variables["x"]!!).getOrNull()
    val y = result.scope.value(result.variables["y"]!!).getOrNull()
    println("  x = $x, y = $y")
}


✓ Optimal solution found!
  Objective value: 56.0
  x = 8, y = 7


## Summary

**What we demonstrated:**

| Component | Purpose |
|-----------|--------|
| `Expr` | Composable expression with operator overloads |
| `sum`, `forAll`, `exists` | Functional aggregations |
| `ConstraintSolverBuilder` | Fluent, functional model definition |
| `SolveResult` | Solution status and value extraction |

**DSL Operators:**

- **Arithmetic**: `+`, `-`, `*`, unary `-`
- **Comparison**: `eq`, `neq`, `lt`, `leq`, `gt`, `geq`
- **Logical**: `and`, `or`, `not`, `implies`
- **Conditional**: `iif(cond, ifTrue, ifFalse)`


## Next Steps

**Build from source:**

```bash
cd kotlin/
./gradlew :solver:shadowJar
```

**Use as a library:**

```kotlin
dependencies {
    implementation("io.github.andreifilonenko:solver:0.1.0")
}
```


## Classic Example: N-Queens Problem

The N-Queens puzzle asks: how can you place N chess queens on an N×N board so that no two queens threaten each other?

This is a classic constraint satisfaction problem with:
- **N decision variables**: one for each column, indicating which row the queen is placed
- **Hard constraints**: 
  - All rows must be different (no two queens on same row)
  - All diagonals must be different (no two queens on same diagonal)


In [6]:
// N-Queens: Place N queens on an NxN board so no two attack each other
val N = 8

val queensModel = ConstraintSolverBuilder()
    .timeLimit(60)
    .numWorkers(4)
    
    // Variables: queens[i] = row of queen in column i
    .variables { scope ->
        (0 until N).associate { col ->
            "q$col" to scope.int("queen_$col", 0, (N - 1).toLong())
        }
    }
    
    // Hard: All rows must be different (no two queens on same row)
    .hard("different_rows") { _, vars ->
        val queens = (0 until N).map { vars["q$it"]!! }
        allDifferent(queens)
    }
    
    // Hard: No two queens on the same diagonal (↘ direction)
    // queens[i] + i must be different for all i
    .hard("different_diag1") { _, vars ->
        val diag1 = (0 until N).map { i -> vars["q$i"]!! + i.toLong() }
        allDifferent(diag1)
    }
    
    // Hard: No two queens on the same diagonal (↗ direction)
    // queens[i] - i must be different for all i
    .hard("different_diag2") { _, vars ->
        val diag2 = (0 until N).map { i -> vars["q$i"]!! - i.toLong() }
        allDifferent(diag2)
    }
    
    .build()

println("$N-Queens model built")


8-Queens model built


In [7]:
// Solve and visualize the board
val queensResult = queensModel.solve()

if (queensResult.hasSolution) {
    println("✓ Solution found for $N-Queens!\n")
    
    // Extract queen positions
    val positions = (0 until N).map { col ->
        queensResult.scope.value(queensResult.variables["q$col"]!!).getOrNull()!!.toInt()
    }
    
    // Print the board
    println("  " + (0 until N).joinToString(" ") { ('a' + it).toString() })
    for (row in 0 until N) {
        print("${N - row} ")
        for (col in 0 until N) {
            print(if (positions[col] == row) "♛ " else "· ")
        }
        println()
    }
    println()
    println("Queen positions (column → row): ${positions.mapIndexed { c, r -> "${('a' + c)}${N - r}" }}")
} else {
    println("✗ No solution found: ${queensResult.status}")
}


✓ Solution found for 8-Queens!

  a b c d e f g h
8 · · ♛ · · · · · 
7 · · · · · · ♛ · 
6 · ♛ · · · · · · 
5 · · · · · · · ♛ 
4 · · · · ♛ · · · 
3 ♛ · · · · · · · 
2 · · · ♛ · · · · 
1 · · · · · ♛ · · 

Queen positions (column → row): [a3, b6, c8, d2, e4, f1, g7, h5]
